### Лекция 11 - Многопоточность и асинхронность в Python

- Модуль threading
- Механизмы синхронизации
- Очереди, Producer/Consumer
- Глобальная блокировка интерпретатора (GIL)
- Асинхронность в Python
- Модуль asyncio

In [1]:
import threading
import time

In [2]:
def prime_numbers(lower, upper):
    ''' (неэффективная) функция для вывода простых чисел в диапазоне '''
    if lower < 2:
        lower = 2
    for num in range(lower, upper + 1):
        for i in range(2, num):
            if num % i == 0:
                break
        else:
            print('- up to {} : pn={}'.format(upper, num))
                

t = threading.Thread(target=prime_numbers,
                     args=(2**20, 2**20 + 256))
t.start()

- up to 1048832 : pn=1048583
- up to 1048832 : pn=1048589
- up to 1048832 : pn=1048601
- up to 1048832 : pn=1048609
- up to 1048832 : pn=1048613
- up to 1048832 : pn=1048627
- up to 1048832 : pn=1048633
- up to 1048832 : pn=1048661
- up to 1048832 : pn=1048681
- up to 1048832 : pn=1048703
- up to 1048832 : pn=1048709
- up to 1048832 : pn=1048717
- up to 1048832 : pn=1048721
- up to 1048832 : pn=1048759
- up to 1048832 : pn=1048783
- up to 1048832 : pn=1048793
- up to 1048832 : pn=1048799
- up to 1048832 : pn=1048807
- up to 1048832 : pn=1048829


In [3]:
class PrimeNumberThread(threading.Thread):
    ''' Класс генерации простых чисел с функцией потока '''
    def __init__(self, lower, upper):
        self._lower = lower
        self._upper = upper
        threading.Thread.__init__(self)

    def run(self):
        for num in range(self._lower, self._upper + 1):
            for i in range(2, num):
                if num % i == 0:
                    break
            else:
                print('- up to {} : pn={}'.format(self._upper, num))


t = PrimeNumberThread(150, 200)
t.start()

- up to 200 : pn=151
- up to 200 : pn=157
- up to 200 : pn=163
- up to 200 : pn=167
- up to 200 : pn=173
- up to 200 : pn=179
- up to 200 : pn=181
- up to 200 : pn=191
- up to 200 : pn=193
- up to 200 : pn=197
- up to 200 : pn=199


In [4]:
# пример одновременного выполнения 2 потоков:
t1 = PrimeNumberThread(2**20, 2**20 + 256)
t2 = PrimeNumberThread(2**21, 2**21 + 256)
t1.start()
t2.start()

- up to 1048832 : pn=1048583
- up to 1048832 : pn=1048589
- up to 2097408 : pn=2097169
- up to 1048832 : pn=1048601
- up to 1048832 : pn=1048609
- up to 2097408 : pn=2097211
- up to 1048832 : pn=1048613
- up to 1048832 : pn=1048627
- up to 2097408 : pn=2097223
- up to 1048832 : pn=1048633
- up to 1048832 : pn=1048661
- up to 2097408 : pn=2097229
- up to 1048832 : pn=1048681
- up to 1048832 : pn=1048703
- up to 2097408 : pn=2097257
- up to 1048832 : pn=1048709
- up to 1048832 : pn=1048717
- up to 2097408 : pn=2097259
- up to 1048832 : pn=1048721
- up to 1048832 : pn=1048759
- up to 2097408 : pn=2097287
- up to 1048832 : pn=1048783
- up to 2097408 : pn=2097289- up to 1048832 : pn=1048793

- up to 1048832 : pn=1048799
- up to 1048832 : pn=1048807- up to 2097408 : pn=2097311

- up to 1048832 : pn=1048829
- up to 2097408 : pn=2097317
- up to 2097408 : pn=2097349
- up to 2097408 : pn=2097373
- up to 2097408 : pn=2097383
- up to 2097408 : pn=2097397
- up to 2097408 : pn=2097401


### Пример окна с индикацией прогресса (wxPython)

![wxPythonDemo](images/gauge_demo.png)

In [ ]:
import wx


class DownloadThread(threading.Thread):
    ''' Класс простого потока индикации прогресса.
         
        _percentage - текущий процент выполнения операции
        _update - функция, которая вызывается 
        _callback - функция, которая будет вызываться
    '''
    def __init__(self, thread_name, thread_func, callback):
        threading.Thread.__init__(self)
        self.daemon = True
        self.name = thread_name
        self._percentage = 0
        self._update = thread_func
        self._callback = callback
    
    def run(self):
        while self._percentage < 100:
            time.sleep(0.03)
            self._percentage += 1
            self._update(self._percentage)
        
        self._callback(self.name)


class MainFrame(wx.Frame):
    def __init__(self):
        wx.Frame.__init__(self, None, wx.ID_ANY, 'Threads!', size=(400, 300))
        
        panel = wx.Panel(self, -1)
        self.gauge1 = wx.Gauge(panel, -1, pos=(20,20), size=(360,40))
        self.gauge2 = wx.Gauge(panel, -1, pos=(20,80), size=(360,40))

        button_start1 = wx.Button(panel, -1, 'start thread #1', pos=(20,140),
                                 size = (170, 50) )
        button_start2 = wx.Button(panel, -1, 'start thread #2', pos=(210,140),
                                 size = (170, 50) )
        self.Bind(wx.EVT_BUTTON, self.on_start_thread1, button_start1)
        self.Bind(wx.EVT_BUTTON, self.on_start_thread2, button_start2)

        self.info_text = wx.StaticText(panel, -1, pos=(20, 230), size=(360, 20))
        self.info_text.SetLabel('Idle...')
        
    def on_start_thread1(self,event):
        self.dthread1 = DownloadThread('1', self.update_gauge1, self.download_finished)
        self.dthread1.start()
        
    def on_start_thread2(self,event):
        self.dthread2 = DownloadThread('2', self.update_gauge2, self.download_finished)
        self.dthread2.start()
        
    def update_gauge1(self, percentage):
        self.gauge1.SetValue(percentage)        
        
    def update_gauge2(self, percentage):
        self.gauge2.SetValue(percentage)
        
    def download_finished(self, thread_no):
        self.info_text.SetLabel('Thread {} has finished'.format(thread_no))


if __name__ == '__main__':
    app = wx.App()
    frame = MainFrame()
    frame.Show()
    app.MainLoop()

### Механизмы синхронизации потоков

- мютексы (mutexes, lock)
- семафоры (semaphores)
- события (events)
- conditions

### События (events)

In [6]:
from random import randint

model = ''

def load_model():
    global model
    models = ['First model', 'Second model', 'Third model', None]
    for m in models:
        views_updated.wait()
        views_updated.clear()
        model = m
        time.sleep(randint(1, 3))
        model_loaded.set()
    
def update_views():
    while True:
        model_loaded.wait()
        model_loaded.clear()
        if model is None:
            break
        for i in range(3):
            print('View {} has been updated: {}'.format(i+1, model))
        views_updated.set()

# инициализируем события
model_loaded = threading.Event()
views_updated = threading.Event()

t1 = threading.Thread(target=load_model)
t2 = threading.Thread(target=update_views)
t1.start()
t2.start()

# устанавливаем "флажок" первого события
views_updated.set()

# присоединяем потоки к основному потоку
# (т.е. не двигаемся дальше, пока не завершатся потоки)
t1.join()
t2.join()

View 1 has been updated: First model
View 2 has been updated: First model
View 3 has been updated: First model
View 1 has been updated: Second model
View 2 has been updated: Second model
View 3 has been updated: Second model
View 1 has been updated: Third model
View 2 has been updated: Third model
View 3 has been updated: Third model


### Механизмы блокировки

In [7]:
# Мютекс (lock)

lock = threading.Lock()

def count_numbers(n, delay):
    with lock:
        print('Printing numbers from 1 to', n)
        for i in range(1, n + 1):
            print(i)
            time.sleep(delay)

t1 = threading.Thread(target=count_numbers, args=(10, 1))
t2 = threading.Thread(target=count_numbers, args=(5, 2))

t1.start()
t2.start()

t1.join() 
t2.join()

Printing numbers from 1 to 10
1
2
3
4
5
6
7
8
9
10
Printing numbers from 1 to 5
1
2
3
4
5


### Producer/Consumer

In [8]:
# Работа с потоками через очередь Queue
# Класс Queue из модуля queue - потокобезопасная очередь
# (с помощью которой можно подавать данные с одного потока на другой).

from queue import Queue


# очередь
q = Queue()
# мютекс, блокирующий вывод на консоль в потоке
print_lock = threading.Lock()


def consumer():
    while True:
        # достаем из очереди задачу
        worker = q.get()
        # делаем что надо
        time.sleep(1)
        with print_lock:
            print('Поток {} обрабатывает задачу {}'.format(
                    threading.current_thread().name, worker))
        # отмечаем, что задача отработана
        q.task_done()

def producer(n):
    # каждый продюсер кладет в очередь 5 задач
    for task in range(5):
        q.put(n*5 + task)
        
# создаем 10 консьюмеров
for _ in range(10):
    t = threading.Thread(target=consumer, daemon=True)
    t.start()

# создаем 4 поставщиков
for i in range(4):
    t = threading.Thread(target=producer, daemon=True, args=(i,))
    t.start()
    
# ждем завершения потока
q.join()

Поток Thread-25 обрабатывает задачу 9
Поток Thread-24 обрабатывает задачу 8
Поток Thread-22 обрабатывает задачу 6
Поток Thread-23 обрабатывает задачу 7
Поток Thread-21 обрабатывает задачу 5
Поток Thread-17 обрабатывает задачу 4
Поток Thread-20 обрабатывает задачу 3
Поток Thread-19 обрабатывает задачу 2
Поток Thread-18 обрабатывает задачу 1
Поток Thread-16 обрабатывает задачу 0
Поток Thread-18 обрабатывает задачу 18
Поток Thread-16 обрабатывает задачу 19
Поток Thread-19 обрабатывает задачу 17
Поток Thread-23 обрабатывает задачу 13
Поток Thread-22 обрабатывает задачу 12
Поток Thread-24 обрабатывает задачу 11
Поток Thread-25 обрабатывает задачу 10
Поток Thread-20 обрабатывает задачу 16
Поток Thread-17 обрабатывает задачу 15
Поток Thread-21 обрабатывает задачу 14


### GIL (Global Interpreter Lock)

Если свести к одному предложению, то: в Python CPU-bound задачи, запрограммированные как многопоточные, по факту не многопоточны. Причиной этому - глобальная блокировка интерпретатора на уровне реализации CPython (семафоры ставятся практически на все участки сишного кода). Так уж сделал Гвидо, и у этого подхода есть серьезные преимущества (однопоточные скрипты работают более эффективно + дополнительная защита потоконебезопасных участков кода на С).

Подробно о внутренностях GIL пишет Девид Бизли:
- http://www.dabeaz.com/python/GIL.pdf (старый GIL, < Python 3.2)
- http://www.dabeaz.com/python/NewGIL.pdf (новый GIL)

Перевод на хабре:
- https://habrahabr.ru/post/84629/

Из-за особенностей GIL + механизмов переключения потоков в любой ОС, код, "разнесенный" по потокам, будет работать даже медленнее, чем однопоточный.

Проверим это:


In [9]:
# Посчитаем сумму всех чисел из диапазона [1, 20000000)
# однопоточная версия
lower = 1
upper = 20000000
 
tm1 = time.time()
 
total = 0
for i in range(lower, upper):
    total += i

tm2 = time.time()
 
print('Single-threaded version: {} sec'.format(tm2 - tm1))
print('Result: {}'.format(total))

Single-threaded version: 1.9972162246704102 sec
Result: 199999990000000


In [10]:
# многопоточная версия

middle = 10000000
 
def count_sum(start, end, res):
    total = 0
    for i in range(start, end):
        total += i
    res[0] = total 


sum1 = [0] 
sum2 = [0]
 
tm1 = time.time()
 
t1 = threading.Thread(target=count_sum, args=(lower, middle, sum1)) 
t2 = threading.Thread(target=count_sum, args=(middle, upper, sum2)) 
t1.start()
t2.start()
t1.join()
t2.join()
 
total = sum1[0] + sum2[0]
 
tm2 = time.time()
 
print('Multi-threaded version: {} sec'.format(tm2 - tm1))
print('Result: {}'.format(total))

Multi-threaded version: 1.2708654403686523 sec
Result: 199999990000000


Согласно GIL, многопоточная версия не должна быть быстрее однопоточной.
А у нас результат оказался противоположным.

Причина - хитрая. В примере выше мы вызываем глобальную функцию, оперирующую глобальными переменными. Глобальные переменные хранятся в хеш-таблице.
А потоки выполняются посредством своих функций со стеком переменных, и там (как и во всяких функциях) переменные хранятся в массивах. Скорость адресации в массиве выше скорости доступа к элементу хеш-таблицы. За счет этого получается выигрыш второй версии.

Ну а если напишем однопоточную версию с помощью функции, то получим ожидаемый результат:

In [11]:
lower = 1
upper = 20000000
 
tm1 = time.time()
 
total = [0]
count_sum(lower, upper, total)
 
tm2 = time.time()
 
print('Single-threaded version: {} sec'.format(tm2 - tm1))
print('Result: {}'.format(total[0]))

Single-threaded version: 1.2537884712219238 sec
Result: 199999990000000


NB. Время замерять часто удобнее с помощью timeit:

In [12]:
%timeit count_sum(lower, upper, total)

1 loop, best of 3: 1.13 s per loop


### Асинхронность в Python

Асинхронность - то, без чего не обходится ни одна нормальная современная технология программирования. Многое из того, что в "лихие 90-ые" за неимением альтернатив делалось в потоках, сейчас лучше и эффективнее делать на асинхронных механизмах, а именно: всякие I/O задачи, где основной поток, выполняясь синхронно, тратит свое время на банальное ожидание ресурса, в то время как он мог бы преспокойно заниматься и другими делами. Примеры: загрузка файлов на локальный компьютер по сетевым протоколам; выполнение мудрёного для СУБД SQL-запроса и ожидание ответа; считывание данных с СОМ-порта с определенной периодичностью, и т.д.

#### Основные понятия асинхронного программирования:

    event loops
    awaitables
    coroutines
    generators
    futures
    concurrent futures
    tasks
    executors
    transports
    protocols

Все это есть в питончике. Читаем также здесь: http://lucumr.pocoo.org/2016/10/30/i-dont-understand-asyncio/

Ниже рассмотрим примеры из официальной документации Python

In [13]:
import asyncio
import datetime

In [14]:
# Демонстрация event_loop
# call_soon, call_later

def display_date(end_time, loop):
    ''' показ времени каждую секунду '''
    print(datetime.datetime.now())
    if (loop.time() + 1.0) < end_time:
        loop.call_later(1, display_date, end_time, loop)
    else:
        loop.stop()

# Получаем себе цикл обработки сообщений (из текущего потока)
loop = asyncio.get_event_loop()

# вызываем функцию display_date()
end_time = loop.time() + 5.0
loop.call_soon(display_date, end_time, loop)

# говорим циклу выполняться вечно (но он прервется вызовом loop.stop() из функции)
loop.run_forever()
# Закрываем цикл обработки сообщений (в jupyter'е я это делать, конечно, не буду)
#loop.close()

2017-05-03 20:26:47.935598
2017-05-03 20:26:48.952208
2017-05-03 20:26:49.959222
2017-05-03 20:26:50.970285
2017-05-03 20:26:51.970992


In [15]:
@asyncio.coroutine
def display_date(loop):
    end_time = loop.time() + 5.0
    while True:
        print(datetime.datetime.now())
        if (loop.time() + 1.0) >= end_time:
            break
        yield from asyncio.sleep(1)

loop = asyncio.get_event_loop()
loop.run_until_complete(display_date(loop))
# Закрываем цикл обработки сообщений (в jupyter'е я это делать, конечно, не буду)
#loop.close()

2017-05-03 20:26:55.055535
2017-05-03 20:26:56.068854
2017-05-03 20:26:57.070970
2017-05-03 20:26:58.073680
2017-05-03 20:26:59.082370


Начиная с Python 3.5, такие конструкции "доведены до ума" и записываются более коротко:

```asyncio.coroutine``` заменяется на ```async def foo()```

```yield from``` заменяется на ```await```

In [16]:
async def display_date(loop):
    end_time = loop.time() + 5.0
    while True:
        print(datetime.datetime.now())
        if (loop.time() + 1.0) >= end_time:
            break
        await asyncio.sleep(1)

loop = asyncio.get_event_loop()
loop.run_until_complete(display_date(loop))
# Закрываем цикл обработки сообщений (в jupyter'е я это делать, конечно, не буду)
#loop.close()

2017-05-03 20:27:02.146636
2017-05-03 20:27:03.153434
2017-05-03 20:27:04.167310
2017-05-03 20:27:05.171045
2017-05-03 20:27:06.184588


In [17]:
"""
Две корутины в цепочке.

Цепочка: compute() -> print_sum().
Корутина print_sum() ожидает окончания compute() перед тем, как вернуть результат.
"""

async def compute(x, y):
    print("Compute %s + %s ..." % (x, y))
    await asyncio.sleep(1.0)
    return x + y

async def print_sum(x, y):
    result = await compute(x, y)
    print("%s + %s = %s" % (x, y, result))

loop = asyncio.get_event_loop()
loop.run_until_complete(print_sum(1, 2))
# цикл обработки сообщений (в jupyter'е я это делать, конечно, не буду)
# loop.close()

Compute 1 + 2 ...
1 + 2 = 3


<img src="https://docs.python.org/3/_images/tulip_coro.png"/>

In [18]:
# Параллельное выполнение трех тасков-задач (tasks),
# каждая из которых представлена корутиной вычисления факториала.

async def factorial(name, number):
    f = 1
    for i in range(2, number+1):
        print("Task %s: Compute factorial(%s)..." % (name, i))
        await asyncio.sleep(1)
        f *= i
    print("Task %s: factorial(%s) = %s" % (name, number, f))

    
loop = asyncio.get_event_loop()
loop.run_until_complete(asyncio.gather(
    factorial("A", 2),
    factorial("B", 3),
    factorial("C", 4),
))
# loop.close()

Task C: Compute factorial(2)...
Task B: Compute factorial(2)...
Task A: Compute factorial(2)...
Task C: Compute factorial(3)...
Task B: Compute factorial(3)...
Task A: factorial(2) = 2
Task C: Compute factorial(4)...
Task B: factorial(3) = 6
Task C: factorial(4) = 24


[None, None, None]

In [19]:
# Пример создания тасков через create_task()

async def reorganize_data(filename):
    # просто пусть длительность паузы зависит от имени файла
    await asyncio.sleep(filename.index('.'))
    print('Reorganized data in file {}'.format(filename))


loop = asyncio.get_event_loop()

reorganize_task1 = loop.create_task(reorganize_data('12.csv'))
reorganize_task2 = loop.create_task(reorganize_data('3.csv'))

loop.run_until_complete(
    asyncio.gather(reorganize_task1, reorganize_task2))
# loop.close()

Reorganized data in file 3.csv
Reorganized data in file 12.csv


[None, None]

In [20]:
# Пример работы с future; task - это подкласс future

async def read_file_contents(future):
    print('Ну, например, читаааааем из большого файла... Ожидайте')
    print()
    # и здесь код чтения из файла, типа await read_file_async()
    await asyncio.sleep(5)
    future.set_result('Прочитали:\nТекст файла... (и далее миллион символов)')

    
future = asyncio.Future()
    
loop = asyncio.get_event_loop()
asyncio.ensure_future(read_file_contents(future))
loop.run_until_complete(future)

print(future.result())
#loop.close()

Ну, например, читаааааем из большого файла... Ожидайте

Прочитали:
Текст файла... (и далее миллион символов)


<hr/>
Ниже рассмотрим, как можно блокирующую функцию (не asyncio-корутину) начать выполнять асинхронно:

In [20]:
import requests

async def download_urls():
    loop = asyncio.get_event_loop()
    future1 = loop.run_in_executor(None,
                                   requests.get,
                                   'https://docs.python.org/3/library/asyncio-task.html')
    future2 = loop.run_in_executor(None,
                                   requests.get,
                                   'http://www.google.com')
    response1 = await future1
    response2 = await future2
    print('first site:')
    print(response1.text[:200])
    print()
    print('second site:')
    print(response2.text[:200])

loop = asyncio.get_event_loop()
loop.run_until_complete(download_urls())
# loop.close()

first site:
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
  "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">


<html xmlns="http://www.w3.org/1999/xhtml">
  <head>
    <meta http-equiv=

second site:
<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="uk"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/goo


In [21]:
from concurrent.futures import ThreadPoolExecutor

# Пул потоков (executor) можем сами создать/настроить
executor = ThreadPoolExecutor(max_workers=3)

async def download_urls(executor):
    loop = asyncio.get_event_loop()
    future1 = loop.run_in_executor(executor,
                                   requests.get,
                                   'https://docs.python.org/3/library/asyncio-task.html')
    future2 = loop.run_in_executor(executor,
                                   requests.get,
                                   'http://www.google.com')
    response1 = await future1
    response2 = await future2
    print('first site:')
    print(response1.text[:200])
    print()
    print('second site:')
    print(response2.text[:200])

    
loop = asyncio.get_event_loop()
loop.run_until_complete(download_urls(executor))
# loop.close()

first site:
<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"
  "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">


<html xmlns="http://www.w3.org/1999/xhtml">
  <head>
    <meta http-equiv=

second site:
<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="uk"><head><meta content="text/html; charset=UTF-8" http-equiv="Content-Type"><meta content="/images/branding/googleg/1x/goo


In [22]:

# Исключения надо ловить обязательно самостоятельно,
# т.к. уведомлений о них никаких не будет
# (разве что если явно спросить у future)

async def suspicious():
    try:
        await some_buggy_operation()
    except ValueError as e:
        print(e)

async def some_buggy_operation():
    raise ValueError('Wow! Exception\n')

    
loop = asyncio.get_event_loop()
loop.run_until_complete(suspicious())
# loop.close()

print('Is loop running?', loop.is_running())

Wow! Exception

Is loop running? False


In [23]:

# корутины можно прерывать (cancellation):

async def poll(incr=1):
    """ бесконечная корутина """
    i = 0
    while True:
        print("Получены новые данные на {} секунде".format(i))
        i += incr
        await asyncio.sleep(incr)
    

async def stop(duration):
    """ Корутина, ожидающая сигнал для остановки цикла обрабокти сообщений """
    await asyncio.sleep(duration)
    # Также можно поднять здесь Exception и прописать
    # `return_when=asyncio.FIRST_EXCEPTION` в `asyncio.wait`.

    
loop = asyncio.get_event_loop()
tasks = [asyncio.ensure_future(poll(2)),
         asyncio.ensure_future(poll(1.5)),
         asyncio.ensure_future(stop(5))]
finished, pending = loop.run_until_complete(
    asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED))

# отменяем остающиеся таски
for task in pending:
    print("Cancelling %s: %s" % (task, task.cancel()))
try:
    loop.run_until_complete(asyncio.gather(*pending))
except asyncio.CancelledError:
    for task in pending:
        print("Cancelled %s: %s" % (task, task.cancelled()))


Получены новые данные на 0 секунде
Получены новые данные на 0 секунде
Получены новые данные на 1.5 секунде
Получены новые данные на 2 секунде
Получены новые данные на 3.0 секунде
Получены новые данные на 4 секунде
Получены новые данные на 4.5 секунде
Cancelling <Task pending coro=<poll() running at <ipython-input-23-46203e8d92bc>:10> wait_for=<Future cancelled>>: True
Cancelling <Task pending coro=<poll() running at <ipython-input-23-46203e8d92bc>:10> wait_for=<Future cancelled>>: True
Cancelled <Task cancelled coro=<poll() done, defined at <ipython-input-23-46203e8d92bc>:4>>: True
Cancelled <Task cancelled coro=<poll() done, defined at <ipython-input-23-46203e8d92bc>:4>>: True


In [24]:

# пример асинхронного варианта паттерна Producer/Consumer

import random

q = asyncio.Queue()


async def producer(num):
    for _ in range(random.randint(1, 5)):
        await q.put(num)
        await asyncio.sleep(random.random())

async def consumer(num):
    while True:
        value = await q.get()
        print('Принял приемщик №{} : значение {}'.format(num, value))


loop = asyncio.get_event_loop()

producers = [loop.create_task(producer(i)) for i in range(7)]
consumers = [loop.create_task(consumer(i)) for i in range(5)]

loop.run_until_complete(asyncio.wait(producers))

for c in consumers:
    c.set_result(True)

Принял приемщик №0 : значение 0
Принял приемщик №0 : значение 1
Принял приемщик №0 : значение 2
Принял приемщик №0 : значение 3
Принял приемщик №0 : значение 4
Принял приемщик №0 : значение 5
Принял приемщик №0 : значение 6
Принял приемщик №0 : значение 6
Принял приемщик №1 : значение 5
Принял приемщик №2 : значение 0
Принял приемщик №3 : значение 3
Принял приемщик №3 : значение 6
Принял приемщик №0 : значение 5
Принял приемщик №1 : значение 1
Принял приемщик №2 : значение 4
Принял приемщик №3 : значение 1
Принял приемщик №4 : значение 1
Принял приемщик №0 : значение 1
Принял приемщик №1 : значение 3
Принял приемщик №2 : значение 4
Принял приемщик №2 : значение 3
Принял приемщик №4 : значение 4
Принял приемщик №0 : значение 3
